In [89]:
library(dplyr)
library(tidyr)

In [90]:
m <- '~/data/awtp2_16S/metadata.tsv'
fgl_data <- '~/data/awtp2_16S/19_0708_KNLK_SD_DNA.csv'
mwq <- '~/data/awtp2_16S/mwq.csv'

In [91]:
#what should it all be called?
NTB<- "90%_conditioned/10%_conventional" #90/10 blends
half<- "45%_conditioned/55%_conventional" #45/55 blends
dpwf<- "100%_conditioned"   #source & merge
mwtp_s<-"100%_conventional" #source & merge

mwtp<- "100%_conventional_water" # influent composition
all_pure<-"100%_conditioned_water"  #influent composition

all_conv<- "100%_conventional" #location_code
dpwf_s<- "100%_conditioned" #location_code

In [92]:
seq_data=read.table(m, header=TRUE, sep="\t")
seq_data$sample_date <- as.Date(seq_data$sample_date, "%m/%d/%Y")
seq_data$location_code<- gsub("Loop_MWTP_inf",all_conv,seq_data$location_code)
seq_data$location_code<- gsub("Loop_DPWF_inf",dpwf_s,seq_data$location_code)
seq_data$location_code<- gsub("DWDS_282B","DWDS_282",seq_data$location_code)
seq_data$location_code<- gsub("WW_3ary","WW_tertiary",seq_data$location_code)

#label all samples as "N" for non-experimental
seq_data$exp_merge<-"N"
#label experimental samples (negative controls and positive controls) as experimental (because these rows won't have water quality data to merge)
seq_data[((seq_data$location_code=="negative_control")| (seq_data$location_code=="positive_control"))|(seq_data$location_code=="field_blank"),]$exp_merge<-"Y"

head(seq_data)

id,id_full,count_raw_reads,barcode_forward,barcode_reverse,plate,well,metagenomic_sample,sample_code_full_and_batch,sample_date,⋯,location_code,sample_type,sampler_name,Ext_batch,Extraction_Date,Ext_num_unique,Microconcentrated.,DNA_Ext_conc_ngperuL,kit,exp_merge
<fct>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<date>,⋯,<chr>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>
SD001,KNLK_SD001,21320,TACGAGAC,GAGCTCAT,1,D2,Y,EXP_0_field_bulk_9_NA_19,2018-04-23,⋯,field_blank,DEUF,Lauren_Kennedy,19,9/8/2018,183,Y,0.126,PowerSoil Pro,Y
SD002,KNLK_SD002,43792,CTACTATA,CTCTAGAG,1,A3,Y,EXP_0_extraction_NA_NA_NA_11,NA,⋯,negative_control,NA,NA,11,7/7/2018,94,Y,0.226,PowerSoil Pro,Y
SD003,KNLK_SD003,52728,CGTTACTA,CTCTAGAG,1,B3,Y,DWDS_F_utility_bulk_9_A_12,2018-03-28,⋯,100%_conventional,DEUF,Lauren_Kennedy,12,7/9/2018,99,Y,0.252,PowerSoil Pro,N
SD004,KNLK_SD004,35359,AGAGTCAC,CTCTAGAG,1,C3,Y,SDS_F_5_biofilm_9_NA_8,2018-04-09,⋯,Loop_5,biofilm,Lauren_Kennedy,8,7/6/2018,51,Y,0.302,PowerSoil Pro,N
SD005,KNLK_SD005,35380,TACGAGAC,CTCTAGAG,1,D3,Y,DWDS_F_328_bulk_5_A_13,2018-04-19,⋯,DWDS_328,DEUF,Lauren_Kennedy,13,8/23/2018,116,Y,0.307,PowerSoil Pro,N
SD006,KNLK_SD006,30734,TACGAGAC,CATGAGGA,1,D5,Y,AWTP_2_RO2_biofilm_sep_1_A_RO1_A57,2018-01-10,⋯,RO_BF_separator,biofilm,Rose_Kantor,RO1,9/13/2018,A57,Y,1.17,Powersoil Pro,N


In [93]:
ext_data <- read.csv(fgl_data, header=TRUE, row.names = NULL)
head(ext_data)

id,buffer,volume_uL,Avg.Size,ng.uL,nM,uL_for_200ng
<fct>,<fct>,<int>,<int>,<dbl>,<dbl>,<dbl>
KNLK_SD185,18 mM TAE bufer,15,385,0.1300,0.5556,359.971202
KNLK_SD003,20 mM tris-HCl with 0.1 mM EDTA,50,393,10.0087,41.8977,4.773532
KNLK_SD004,20 mM tris-HCl with 0.1 mM EDTA,50,390,7.1627,30.2149,6.619251
KNLK_SD005,20 mM tris-HCl with 0.1 mM EDTA,50,390,19.2228,81.0844,2.466566
KNLK_SD006,20 mM tris-HCl with 0.1 mM EDTA,50,393,12.7806,53.5584,3.734242
KNLK_SD007,20 mM tris-HCl with 0.1 mM EDTA,50,396,36.0204,149.5023,1.337772


In [94]:
wq_data <- read.csv(mwq, header=TRUE, row.names = NULL)
wq_data <- select(wq_data, select=-X)
wq_data$sample_date <- as.Date(wq_data$sample_date, "%Y-%m-%d")
wq_data$location_code <- as.character(wq_data$location_code)
wq_data$location_code<- gsub("WW_Tertiary","WW_tertiary",wq_data$location_code)
wq_data$location_code<- gsub("MWTP_inf",all_conv,wq_data$location_code)
wq_data$location_code<- gsub("DPWF_inf",dpwf_s,wq_data$location_code)
wq_data<-subset(wq_data, select=-c(project_type, merge_id))
wq_data=subset(wq_data, is.na(wq_data$experimental_detail))
wq_data$exp_merge<-"N" #sample date and location code shared by some samples so making it easier to merge seq_data

head(wq_data)

free_Cl2_mg.L,total_Cl2_mg.L,pH,temp_C,disinfectant,sample_date,broad_location,location_code,HPC_avg_MPN_per_100mL,experimental_detail,percent_CRA,SG,SGPI,intra_ATP_avg_nM,total_ATP_avg_nM,extra_ATP_avg_nM,exp_merge
<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<date>,<fct>,<chr>,<int>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
0.02,0.07,8.37,10.9,Chloramine,2018-03-19,Pipe_Loop,Loop_2,NA,NA,NA,0.0,5475.7,0.0001058645,1.135176e-03,0.0009924353,N
NA,NA,NA,NA,NA,2017-09-14,AWTP_2,AOP,NA,NA,NA,67.7,3.3,NA,4.878450e-05,NA,N
NA,NA,NA,NA,NA,2017-11-14,AWTP_2,AOP,NA,NA,NA,85.7,26.3,NA,8.604244e-05,NA,N
0.01,0.05,8.32,11.1,Chloramine,2018-03-19,Pipe_Loop,Loop_1,NA,NA,NA,96.0,227.7,0.0009927931,1.816073e-03,0.0007063832,N
NA,NA,NA,NA,NA,2017-12-14,AWTP_2,AOP,NA,NA,NA,97.5,6.6,NA,4.177241e-05,NA,N
NA,NA,NA,NA,NA,2017-12-14,AWTP_2,RO_3stage,NA,NA,NA,97.8,69.8,NA,1.756935e-04,NA,N


In [95]:
table_data <- merge(seq_data, ext_data, by.x='id_full', by.y='id')
table_data <- left_join(table_data,wq_data)
head(table_data)

Joining, by = c("sample_date", "location_code", "exp_merge")


id_full,id,count_raw_reads,barcode_forward,barcode_reverse,plate,well,metagenomic_sample,sample_code_full_and_batch,sample_date,⋯,disinfectant,broad_location,HPC_avg_MPN_per_100mL,experimental_detail,percent_CRA,SG,SGPI,intra_ATP_avg_nM,total_ATP_avg_nM,extra_ATP_avg_nM
<fct>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<date>,⋯,<fct>,<fct>,<int>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
KNLK_SD003,SD003,52728,CGTTACTA,CTCTAGAG,1,B3,Y,DWDS_F_utility_bulk_9_A_12,2018-03-28,⋯,Chloramine,DWDS_F,26,NA,0.4626015,26076.8,471.2,NA,NA,NA
KNLK_SD004,SD004,35359,AGAGTCAC,CTCTAGAG,1,C3,Y,SDS_F_5_biofilm_9_NA_8,2018-04-09,⋯,Chloramine,Pipe_Loop,NA,NA,NA,5999.9,1786.3,7.156732e-05,0.001976834,0.0018942843
KNLK_SD005,SD005,35380,TACGAGAC,CTCTAGAG,1,D3,Y,DWDS_F_328_bulk_5_A_13,2018-04-19,⋯,Chloramine,DWDS_F,17,NA,0.6899480,1946.3,768.3,1.629695e-01,0.163264704,0.0002094437
KNLK_SD006,SD006,30734,TACGAGAC,CATGAGGA,1,D5,Y,AWTP_2_RO2_biofilm_sep_1_A_RO1_A57,2018-01-10,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
KNLK_SD007,SD007,35135,CTACTATA,AGTCTAGA,1,A6,Y,AWTP_3_MF_tank_bulk_1_A_18,2018-03-13,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
KNLK_SD008,SD008,16805,CGTTACTA,AGTCTAGA,1,B6,Y,DWDS_F_328_bulk_1_A_12,2018-02-14,⋯,Chloramine,DWDS_F,6,NA,0.2500000,12870.7,4390.4,5.347541e-04,0.001805507,0.0012105474


In [96]:
rownames(table_data) <- table_data$id_full

#make sampledate column a date
table_data$sample_date <- as.Date(table_data$sample_date, "%m/%d/%Y")
table_data$Extraction_Date <- as.Date(table_data$Extraction_Date, "%m/%d/%Y")
table_data$batch_sample_date <- as.Date(table_data$batch_sample_date, "%m/%d/%Y") 
table_data$SampleID <- row.names(table_data )

#make numeric cols numeric
table_data$total_Cl2_mg.L<-as.numeric(table_data$total_Cl2_mg.L)
table_data$free_Cl2_mg.L<-as.numeric(table_data$free_Cl2_mg.L)
table_data$pH<-as.numeric(table_data$pH)
table_data$temp_C<-as.numeric(table_data$temp_C)
table_data$percent_CRA<-as.numeric(table_data$percent_CRA)
table_data$HPC_avg_MPN_per_100mL<-as.numeric(table_data$HPC_avg_MPN_per_100mL)
table_data$SG<-as.numeric(table_data$SG)
table_data$SGPI<-as.numeric(table_data$SGPI)
table_data$total_ATP_avg_nM<-as.numeric(table_data$total_ATP_avg_nM)
table_data$intra_ATP_avg_nM<-as.numeric(table_data$intra_ATP_avg_nM)
table_data$extra_ATP_avg_nM<-as.numeric(table_data$extra_ATP_avg_nM)

#No wq data taken for pipe loops batch end on 10/23 or 2/19

In [97]:
head(table_data)

,id_full,id,count_raw_reads,barcode_forward,barcode_reverse,plate,well,metagenomic_sample,sample_code_full_and_batch,sample_date,⋯,broad_location,HPC_avg_MPN_per_100mL,experimental_detail,percent_CRA,SG,SGPI,intra_ATP_avg_nM,total_ATP_avg_nM,extra_ATP_avg_nM,SampleID
,<fct>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<date>,⋯,<fct>,<dbl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
KNLK_SD003,KNLK_SD003,SD003,52728,CGTTACTA,CTCTAGAG,1,B3,Y,DWDS_F_utility_bulk_9_A_12,2018-03-28,⋯,DWDS_F,26,NA,0.4626015,26076.8,471.2,NA,NA,NA,KNLK_SD003
KNLK_SD004,KNLK_SD004,SD004,35359,AGAGTCAC,CTCTAGAG,1,C3,Y,SDS_F_5_biofilm_9_NA_8,2018-04-09,⋯,Pipe_Loop,NA,NA,NA,5999.9,1786.3,7.156732e-05,0.001976834,0.0018942843,KNLK_SD004
KNLK_SD005,KNLK_SD005,SD005,35380,TACGAGAC,CTCTAGAG,1,D3,Y,DWDS_F_328_bulk_5_A_13,2018-04-19,⋯,DWDS_F,17,NA,0.6899480,1946.3,768.3,1.629695e-01,0.163264704,0.0002094437,KNLK_SD005
KNLK_SD006,KNLK_SD006,SD006,30734,TACGAGAC,CATGAGGA,1,D5,Y,AWTP_2_RO2_biofilm_sep_1_A_RO1_A57,2018-01-10,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,KNLK_SD006
KNLK_SD007,KNLK_SD007,SD007,35135,CTACTATA,AGTCTAGA,1,A6,Y,AWTP_3_MF_tank_bulk_1_A_18,2018-03-13,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,KNLK_SD007
KNLK_SD008,KNLK_SD008,SD008,16805,CGTTACTA,AGTCTAGA,1,B6,Y,DWDS_F_328_bulk_1_A_12,2018-02-14,⋯,DWDS_F,6,NA,0.2500000,12870.7,4390.4,5.347541e-04,0.001805507,0.0012105474,KNLK_SD008


In [99]:
table_data <- table_data[,c("id",setdiff(names(table_data),"id"))]
head(table_data)
write.table(table_data, '~/data/awtp2_16S/metadata_final.tsv', sep="\t", quote=FALSE, row.names=FALSE)

,id,id_full,count_raw_reads,barcode_forward,barcode_reverse,plate,well,metagenomic_sample,sample_code_full_and_batch,sample_date,⋯,broad_location,HPC_avg_MPN_per_100mL,experimental_detail,percent_CRA,SG,SGPI,intra_ATP_avg_nM,total_ATP_avg_nM,extra_ATP_avg_nM,SampleID
,<fct>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<date>,⋯,<fct>,<dbl>,<lgl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
KNLK_SD003,SD003,KNLK_SD003,52728,CGTTACTA,CTCTAGAG,1,B3,Y,DWDS_F_utility_bulk_9_A_12,2018-03-28,⋯,DWDS_F,26,NA,0.4626015,26076.8,471.2,NA,NA,NA,KNLK_SD003
KNLK_SD004,SD004,KNLK_SD004,35359,AGAGTCAC,CTCTAGAG,1,C3,Y,SDS_F_5_biofilm_9_NA_8,2018-04-09,⋯,Pipe_Loop,NA,NA,NA,5999.9,1786.3,7.156732e-05,0.001976834,0.0018942843,KNLK_SD004
KNLK_SD005,SD005,KNLK_SD005,35380,TACGAGAC,CTCTAGAG,1,D3,Y,DWDS_F_328_bulk_5_A_13,2018-04-19,⋯,DWDS_F,17,NA,0.6899480,1946.3,768.3,1.629695e-01,0.163264704,0.0002094437,KNLK_SD005
KNLK_SD006,SD006,KNLK_SD006,30734,TACGAGAC,CATGAGGA,1,D5,Y,AWTP_2_RO2_biofilm_sep_1_A_RO1_A57,2018-01-10,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,KNLK_SD006
KNLK_SD007,SD007,KNLK_SD007,35135,CTACTATA,AGTCTAGA,1,A6,Y,AWTP_3_MF_tank_bulk_1_A_18,2018-03-13,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,KNLK_SD007
KNLK_SD008,SD008,KNLK_SD008,16805,CGTTACTA,AGTCTAGA,1,B6,Y,DWDS_F_328_bulk_1_A_12,2018-02-14,⋯,DWDS_F,6,NA,0.2500000,12870.7,4390.4,5.347541e-04,0.001805507,0.0012105474,KNLK_SD008
